<a href="https://colab.research.google.com/github/kiwa-debug/IPC-predict-system-using-NLP/blob/main/IPC_FInal_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 KB 4.8 MB/s eta 0:00:00


In [2]:
#Importing the Library
import xlsxwriter
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras import layers
from keras.models import load_model
from openpyxl import load_workbook
import tensorflow_hub as hub
#Importing Universal sentence encoder
tf_hub_embedding_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        trainable=False,
                                        name="universal_sentence_encoder")

In [3]:
# Creating Model
def IPC_Self_learning_Model(text):
  train_data=pd.read_excel("ipc_data_1.xlsx")
  X = train_data["Offence"].to_list()
  X = np.array(X)
  train_data_1= train_data['Section']
  train_data_1 = train_data_1.apply(lambda x: ast.literal_eval(x))

  multilabel = MultiLabelBinarizer()
  y = multilabel.fit_transform(train_data_1)


  # Define feature extractor model using TF Hub layer
  inputs = layers.Input(shape=[], dtype=tf.string)
  pretrained_embedding = tf_hub_embedding_layer(inputs) # tokenize text and create embedding
  x = layers.Dense(128, activation="relu")(pretrained_embedding) # add a fully connected layer on top of the embedding
  # Note: you could add more layers here if you wanted to
  outputs = layers.Dense(len(multilabel.classes_), activation="softmax")(x) # create the output layer
  model_1 = tf.keras.Model(inputs=inputs,
                          outputs=outputs)
  


  # Compile the model
  model_1.compile(loss="categorical_crossentropy",
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=["accuracy"])

  model_1.fit(X,  y,
               epochs=30,
                verbose=0)
  
  text = np.array([text])
  model_1_pred_probs = model_1.predict(text)
  n=2
  indices_2 = (-model_1_pred_probs).argsort()[:n]
  q=[]
  for r in range(0,4):
    q.append(multilabel.classes_[indices_2[0,r]])
  

  return q

In [8]:
offence_ ="Murder and Riot" #Enter any offence u want

In [9]:
#Prdict the ipc on given offence
pred_ipc = IPC_Self_learning_Model(offence_)

1/1 [==============================] - 0s 430ms/step


In [6]:
#Making a function so it can return the detail of given ipc and there punisgment
data={}
train_data=pd.read_excel("ipc_data_1.xlsx")
Off = train_data["Offence"]

punish = train_data["Punishment"]

train_data_1= train_data['Section']
train_data_1 = train_data_1.apply(lambda x: ast.literal_eval(x))

Sec_1= train_data_1

for x in range(0,382):
  data[Sec_1[x][0]]=[Off[x],punish[x]]

def data_retrival(code):
  detail = []
  punishment = []
  for i in code:  
    detail.append(data[i][0])
    punishment.append(data[i][1]);
  for j in range(len(code)):
    print(f"{code[j]}\n Detail:{detail[j]}\n Punishment:{punishment[j]}")

In [11]:
#Predicted IPC with there detail and punishment
data_retrival(pred_ipc)

302 IPC
 Detail:Murder
 Punishment:Death, or imprisonmentfor life, and fine
147 IPC
 Detail:Rioting
 Punishment:Imprisonment for 2 years, or fine, or both
148 IPC
 Detail:Rioting armed with a deadly weapon.
 Punishment:Imprisonment for 3 years, or fine, or both.
396 IPC
 Detail:Murder in dacoity
 Punishment:Death, imprisonment for life, or rigorous imprisonment for 10 years and fine
